# DFA Learning

![logo](https://iscasmc.ios.ac.cn/roll/lib/exe/fetch.php?media=wiki:logo.png)

This is a tutorial for the Java Library [```Regular Omega Language Learning (ROLL)```](https://iscasmc.ios.ac.cn/roll/doku.php) in a Groovy kernel Jupyter Notebook.
Groovy is very similar to Java and you can write all code in Java syntax.

**Tips** : If something goes strange, use the menu bar above ```Kernel -> Restart``` to reboot this notebook and run following code again.

---

**First of all, load the jar file of the learning library ROLL.**

In [1]:
%classpath add jar ROLL.jar

Added jar: [ROLL.jar]


In the active automata learning setting proposed by Angluin, there are a `teacher`, which knows the target language $L$, and a `learner`, whose task is to learn the target language, represented by an automaton, from the teacher by means of two kinds of queries: `membership queries` and `equivalence queries`. 
A membership query $MQ[w]$ asks whether a string $w$ belongs to $L$ while an equivalence query $EQ[A]$ asks whether the hypothesis automaton $A$ recognizes $L$. 
The teacher replies with a witness if the hypothesis is incorrect otherwise the learner completes its job.

In the following, we introduce two ways to learn the regular language  $L =\{s \in \Sigma^* \mid b \text{ occurs } 4 n + 3 \text{ times in } s \text{ (} n \ge 0 \text{)} \}$ over the alphabet $\Sigma = \{a, b\}$.

The first way is to use embedded [DK package](http://www.brics.dk/automaton/) to play as the teacher.
The second way is to allow youself to play as the teacher.

**1. Learning the regular language $L$ from DK by giving a target DFA $D$**

we first need to create the target DFA $D$ which accepts the regular language $L$.

In [2]:
import roll.words.Alphabet
import roll.automata.DFA
// you can always import all the classes in roll.jupyter package
import roll.jupyter.*
import java.util.List
import java.util.ArrayList

// in order to create an alphabet, you need an array of Characters
// the variable apList is local since there is type in front of it
List<Character> apList = new ArrayList<Character>();

// in Groovy, we have to do strong cast for Characters 
apList.add((char)'a');
apList.add((char)'b');

// create an alphabet with a Character list
// the created alphabet is global in this notebook
JupyterROLL.createAlphabet(apList);

// use JupyterROLL to create a DFA object D
// the variable target is global since there is no type in front of it
// so we can use this variable everywhere in this notebook
D = JupyterROLL.createDFA();

// now we can get the alphabet in the DFA
alphabet = D.getAlphabet();

[0->a, 1->b]

In [3]:
// now we are ready to create the DFA which accepts the regular language L
// we first create 4 states
D.createState();
D.createState();
D.createState();
D.createState();


// 4 indices for the states
int fst = 0, snd = 1, thd = 2, fur = 3;
// the function getState is to get a state object by its state index
D.getState(fst).addTransition(alphabet.indexOf((char)'a'), fst); // 0 -> 0 via a
D.getState(fst).addTransition(alphabet.indexOf((char)'b'), snd); // 0 -> 1 via b
D.getState(snd).addTransition(alphabet.indexOf((char)'a'), snd); // 1 -> 1 via a
D.getState(snd).addTransition(alphabet.indexOf((char)'b'), thd); // 1 -> 2 via b
D.getState(thd).addTransition(alphabet.indexOf((char)'a'), thd); // 2 -> 2 via a
D.getState(thd).addTransition(alphabet.indexOf((char)'b'), fur); // 2 -> 3 via b
D.getState(fur).addTransition(alphabet.indexOf((char)'a'), fur); // 3 -> 3 via a
D.getState(fur).addTransition(alphabet.indexOf((char)'b'), fst); // 3 -> 0 via b

// set 0 as the initial state
D.setInitial(fst);
// set 3 as a final state
D.setFinal(fur);

// now we can output target in a DOT graph
D

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 1->1 --> 1->1 a <!-- 2 --> 2 2 <!-- 1->2 --> 1->2 b <!-- 2->2 --> 2->2 a <!-- 3 --> 3 3 <!-- 2->3 --> 2->3 b <!-- 3->0 --> 3->0 b <!-- 3->3 --> 3->3 a <!-- 4 --> 4 <!-- 4->0 --> 4->0

Now we are ready to create a DFA learner to learn a hypothesis DFA $A$ from DK. The DFA $A$ accepts the regular language $L$ and has the minimal number of states.

We have 3 types of algorithms and 2 types of data structures to store the membership results during the learning process.

|                              | tree | table |
|:----------------------------:|:----------------------:|:-----------------------:|
| lstar  | ✖️                     | ✔️                      |
| kv     | ✔️                     | ✖️                      |
| column | ✔️                     | ✔️                      |

In the following, we will demostrate how to use the table-based column learning algorithm to show how to learn the target language $L$.

In [4]:
import roll.jupyter.*
    
// we create a global variable sequence which stores the learning procedure as a
// list of Triple object, the Triple object has three elements
// the first is the table/tree data structure, the second is the current hypothesis DFA, and the third is the counterexample
// which refines the previous hypothesis DFA to the current hypothesis
sequence = JupyterROLL.learningSeq("column", "table", D);

// sequence is a java.util.List instance
sequence.size()

3

From the output of the learning list, the target language $L$ has been learned by the learning algorithm with only 3 equivalence queries.

we now can check the Triple object at each step of the learning procedure

In [5]:
// initial learner data
sequence.get(0)

Learner,Hypothesis,Counterexample
|| ϵ | ======== ϵ || - | ======== a || - | b || - |,"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?><!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 -->%3<!-- 0 -->00<!-- 0->0 -->0->0b<!-- 0->0 -->0->0a<!-- 1 -->1<!-- 1->0 -->1->0",


In [6]:
// we get a new hypothesis after one counterexample refinement
sequence.get(1)

Learner,Hypothesis,Counterexample
|| ϵ | bb | ============== ϵ || - | - | b || - | + | ============== a || - | - | ba || - | + | bb || - | - |,"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?><!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 -->%3<!-- 0 -->00<!-- 0->0 -->0->0a<!-- 1 -->11<!-- 0->1 -->0->1b<!-- 1->0 -->1->0b<!-- 1->1 -->1->1a<!-- 2 -->2<!-- 2->0 -->2->0",$bbb$


In [7]:
// yet another hypothesis after a counterexample refinement
sequence.get(2)

Learner 
 Hypothesis 
 Counterexample 
 
 
 || ϵ | bb | b | 
====================
ϵ || - | - | - | 
b || - | + | - | 
bb || - | - | + | 
bbb || + | - | - | 
====================
a || - | - | - | 
ba || - | + | - | 
bba || - | - | + | 
bbba || + | - | - | 
bbbb || - | - | - | 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 1->1 --> 1->1 a <!-- 2 --> 2 2 <!-- 1->2 --> 1->2 b <!-- 2->2 --> 2->2 a <!-- 3 --> 3 3 <!-- 2->3 --> 2->3 b <!-- 3->0 --> 3->0 b <!-- 3->3 --> 3->3 a <!-- 4 --> 4 <!-- 4->0 --> 4->0 
 $bbb$

**We have just learned how to learn a minimal DFA A from DK out of a given DFA D. Sometimes we may not have the target DFA in hand but we know exactly the language we want to learn in mind. ** 

In this case, we can first specify what kind of strings really belong to the target language $L$ and then refine the hypothesis if it does not recognize the target language by ourselves.
We are going to use the tree-based column learning algorithm to show how to learn the DFA A from ourselves.

**2. Learning the regular language $L$ in an interactive way**

In [8]:
import roll.jupyter.*;
import java.util.function.Function;
import roll.words.*;

// now we define a function :: string -> boolean and this function is used to 
// determine whether a string is in the target language
// this function resolves all membership queries posed by the learners
mqOracle = {
    s -> 
    // 4n + 3 -> b
    int num = 0;
    for (int i = 0; i < s.length(); i++) {
        if (s.charAt(i) == 'b') {
                num ++;
        }
    }
    // if the number of b's in s is 4n + 3
    // then s is in the target language
    if(num % 4 == 3) return true;
    return false;
};

// now we create a tree-based column learner to learn the target language 
dfaLearner = JupyterROLL.createDFALearner("column", "tree", mqOracle);
// we can also see the tree data structure of the learner in a DOT graph
dfaLearner

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: BST Pages: 1 --> BST <!-- 0 --> 0 ϵ

In [9]:
// output current hypothesis to see whether it recognizes the target language
dfaLearner.getHypothesis()

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 0->0 --> 0->0 a <!-- 1 --> 1 <!-- 1->0 --> 1->0

In [10]:
// the hypothesis is no correct and we can use a counterexample
// which is in the symmetric difference of the language of A and the target language
// here we use bbb
dfaLearner.refineHypothesis("bbb")
dfaLearner.getHypothesis()

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 1->0 --> 1->0 b <!-- 1->1 --> 1->1 a <!-- 2 --> 2 <!-- 2->0 --> 2->0

In [11]:
// hypothesis is still not correct, use bbb to refine it
dfaLearner.refineHypothesis("bbb")
dfaLearner.getHypothesis()

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 b <!-- 2->0 --> 2->0 b <!-- 2->2 --> 2->2 a <!-- 1 --> 1 1 <!-- 1->0 --> 1->0 b <!-- 1->1 --> 1->1 a <!-- 3 --> 3 <!-- 3->0 --> 3->0

In [12]:
// hypothesis is still not correct, use bbb to refine it
dfaLearner.refineHypothesis("bbb")
dfaLearner.getHypothesis()

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 b <!-- 2->2 --> 2->2 a <!-- 3 --> 3 3 <!-- 2->3 --> 2->3 b <!-- 1 --> 1 1 <!-- 1->0 --> 1->0 b <!-- 1->1 --> 1->1 a <!-- 3->1 --> 3->1 b <!-- 3->3 --> 3->3 a <!-- 4 --> 4 <!-- 4->0 --> 4->0

In [13]:
// hypothesis is now correct, if we use bbb to refine it, the learner will report error message
dfaLearner.refineHypothesis("bbb")

Invalid counterexample, both in hypothesis and target


null